In [6]:
import os
import pandas as pd

In [7]:
PATH='/home/micha/SIM-PhD-King/ubiquitin/atomistic/ub.N25CP.D32CP.D58CP_pH7_CaCl2-0.075M_charmm/simulation/'
mdp_file = "input.0.lammps"

In [166]:
dict_selection = {
    # keyword    : [index_name, index_value, ?index_value_end?]
    'atom_style'   : [0,1],    # atom_style
    'kspace_style' : [0,1,None], # style + accuracy
    'newton'       : [0,1],    # on / off
    'units'        : [0,1],    # unit
    'variable'     : [1,3],    
}
class Lammps_Parameter:
    def __init__(self,**kwargs):
        """
        
        Parameters
        ----------
        kwargs : dict
        
        kwargs
        ------
        fname : str
            filename of the input file
        """
        # set attributes
        for k,v in kwargs.iteritems():
            setattr(self,k,v)
        
        if hasattr(self, 'fname'):
            self.read_file(self.fname)

    def __repr__(self):
        STR="{}(\n".format(self.__class__.__name__)
        for k,v in vars(self).iteritems():
            if type(k) == str:
                STR+=" {}={},\n".format(k,repr(v))
        STR+=")"
        return STR
    
    def __str__(self):
        return str(vars(self))
    
    def read_file(self,path_to_mdp_file):
        """
        Function to read a input file
        """
        dict_variables=dict()
        # Open the file
        with open(path_to_mdp_file) as fp:
            # iterate of the file
            for line in fp:
                line_strip=line.strip()  # strip lines for easy processing
                if len(line_strip) == 0: continue  # skip empty lines
                if line_strip[0] == '#': continue  # skip comments
                line_split=line_strip.split() 
                # interpret the line split
                entry = self.interpret_line_split(line_split)
                # if the value is not None update the dict
                if entry is not None:
                    dict_variables.update(entry)  

        # set attributes           
        for k,v in dict_variables.iteritems():
            setattr(self,k,v)
            
    def interpret_line_split(self,line_split):
        """
        Function to interpret the line_split

        Return
        ------
            dict_entry : dict or None
                Dictionary Entry or None if not in the list
        """

        # key to look for
        key = line_split[0]
        if key in dict_selection.keys():  # check if the key is in there
            # get the indicies
            indicies = dict_selection[key]
            # get the name and the properties
            if len(indicies) == 2:
                index_name, index_value = indicies 
                name = line_split[index_name]
                prop = self.convert_value_type(line_split[index_value])
            elif len(indicies) == 3:
                index_name, index_value, index_value_end  = indicies
                # check for comments:
                if index_value_end is None and '#' in line_split:
                    # reset index_value
                    index_value_end = line_split.index("#")
                name = line_split[index_name]
                prop = [self.convert_value_type(i) for i in line_split[index_value:index_value_end]]
            # return them in form of a dictionary
            return {name : prop}
        else:  # if the key is not defined return None
            return None
    
    def convert_value_type(self,value):
        """
        Function to convert the type of the value
        """
        
        if type(value) == str:
            try:
                tmp = float(value)
                if tmp.is_integer():
                    return int(value)
                else:
                    return float(value)
            except ValueError:
                return value
        elif type(value) == list:
            return [self.convert_value_type(i) for i in value]
        elif type(value) == tuple:
            return (self.convert_value_type(i) for i in value)
        else:
            return value

In [167]:
path_to_mdp_file = os.path.join(PATH,mdp_file)
para_mini = Lammps_Parameter(fname=path_to_mdp_file)    

In [169]:
para_mini

Lammps_Parameter(
 deltat=10,
 atom_style='full',
 newton='on',
 etol=1e-06,
 eramp='100.0*(${nsteps}+${nequil}-step+1)/${nsteps}',
 ntraj=1000,
 n='${shrink}',
 ftol=1e-06,
 ntraj_s=-100,
 prel=1000,
 vReadData=0,
 fffile='ff-charmm_ubiquitin.N25CP.D32CP.D58CP_CaCl2.lammps',
 ts=1,
 plumed=0,
 ens=2,
 fname='/home/micha/SIM-PhD-King/ubiquitin/atomistic/ub.N25CP.D32CP.D58CP_pH7_CaCl2-0.075M_charmm/simulation/input.0.lammps',
 minimise=1,
 units='real',
 vBalance=0,
 shrink=0,
 inpfile='system_start.data',
 relax=1,
 str_dbg="'",
 temp_f=300,
 nthermo=1000,
 resfile='final_restart.$(v_run_no',
 nequil=50000,
 kspace_style=['pppm', 1e-05],
 mtraj=-1,
 temp_s=200,
 dbg_erg=0,
 md=0,
 tscale=1,
 trel=100,
 maxiter=100,
 nsteps=100000,
 pres='1.0',
 str_ens="'",
 str_basic="'step",
 npttype='iso',
)

In [144]:
a='1a'

In [160]:
cat 

ValueError: could not convert string to float: DA